# Spark SQL and DataFrames

### 1. Basic manipulations
First, let's create some users:

In [3]:
from datetime import date

# Let's first create some users:
col_names = ["first_name", "last_name", "birth_date", "gender", "country"]
users = [
  ("Alice", "Jones", date(1981, 4, 15), "female", "Canada"),
  ("John", "Doe", date(1951, 1, 21), "male", "USA"),
  ("Barbara", "May", date(1951, 9, 1), "female", "Australia"),
  ("James", "Smith", date(1975, 7, 12), "male", "United Kingdom"),
  ("Gerrard", "Dupont", date(1968, 5, 9), "male", "France"),
  ("Amanda", "B.", date(1988, 12, 16), "female", "New Zeland")
]

users_df = spark.createDataFrame(users, col_names)
display(users_df) # Only works in Databricks. Elswehere, use "df.show()" or "df.toPandas()"

Now it's your turn, create more users (at least 3, with different names) and add to the initial users, saving the result in a new variable.

In [5]:
new_users_df = ???
all_users_df = ???
display(all_users_df) # or all_users_df.show()

Now, select only two columns and show the resulting DataFrame, without saving it into a variable.

In [7]:
all_users_df.???

Now, register your DataFrame as a table and select the same two columns with a SQL query string

In [9]:
query_string = "???"
all_users_df.??? # creates a local temporary table accessible by a SQL query
spark.sql(query_string).show()

Now we want to add an unique identifier for each user in the table. There are many strategies for that, and for our example we will use the string `{last_name}_{first_name}`

You can use `all_users_df` since your latest operation did not override its values. Add a new column called `user_id` to your DataFrame and save to a new variable.

**Hint:** The first place to look is in the [functions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) package

In [11]:
from pyspark.sql import functions as fn

users_with_id = ???
display(users_with_id)

You can also do the same thing with an User Defined Function by passing a lambda, although it is not recommended when there is already a function in the `functions` package.

Add a new column called `user_id_udf` to the DataFrame, using an UDF that receives two parameters and concatenate them.

In [13]:
concat_udf = fn.udf(???)
users_with_id_udf = ???
display(users_with_id_udf)

Now, let's add another column called `age` with the computed age (in years) of each user, based on a given reference date, and save the resulting DataFrame into a new variable.

**Hint:** You can first compute the age in months, and then divide by 12. A final operation will probably be needed to get an integer number.

In [15]:
reference_date = date(2017, 12, 31)

users_with_age = ???
display(users_with_age)

Now, an analytical question: How many users of each gender who are more than 40 years old exist in this data? The solution must be a DataFrame with two columns: `age` and `count` and two lines, one for each gender.

Bonus: Try to do your solution in a single chain (without intermediate variables)

**Hint:** You will need to filter and aggregate the data.

In [17]:
result = ???
display(result)

### 2. Reading files, performing joins, and aggregating

For this section you will use some fake data of two datasets: `Users` and `Donations`. The data is provided in two CSV files *with header* and using *comma* as separator.

The `Users` dataset contains information about about the users. 

The `Donations` dataset contains information about Donations performed by those users.

The first task is to read these files into the appropriate DataFrames.

**Note:** You need to set the option "inferSchema" to true in order to have the columns in the correct types.

_The data for this section has been created using [Mockaroo](https://www.mockaroo.com/)_.

In [19]:
users_from_file = spark.read.???
donations = spark.read.???

Now investigate the columns, contents and size of both datasets

In [21]:
# print the column names and types
users_from_file.???
donations.??? 

# print 5 elements of the datasets in a tabular format
users_from_file.???
donations.???

# print the number of lines of each dataset
print users_from_file.???
print donations.???

**Note:** If all the column types shown in the previous results are "string", you need to make sure you passed "inferSchema" as true when reading the CSV files before continuing.

Before using the data, we may want to add some information about the users. 

Add a column containing the age of each user.

In [23]:
users_from_file = users_from_file.???
users_from_file.show()

Another useful information to have is the age **range** of each user. Using the `when` function, create the following 5 age ranges:
- "(0, 25]"
- "(25, 35]"
- "(35, 45]"
- "(45, 55]"
- "(55, ~]"

And add a new column to the users DataFrame, containing this information.

**Note:** When building logical operations with Spark DataFrames, it's better to be add parantheses. Example:
```python
df.select("name", "age").where( (df("age") > 20) & (df("age") <= 30) )
```

**Note 2:** If you are having problems with the `when` function, you can make an User Defined Function and do your logic in standard python.

In [25]:
users_from_file = users_from_file.???
users_from_file.show()

Now that we have improved our users' DataFrame, the first analysis we want to make is the average donation performed by each gender. However, the gender information and the donation value are in different tables, so first we need to join them, using the `user_id` as joining key.

**Note:** Make sure you are not using the `users` DataFrame from the first part of the lab.

**Note 2:** For better performance, you can [broadcast](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.broadcast) the smaller dataset. But you should only do this when you are sure the DataFrame fits in the memory of the nodes.

In [27]:
joined_df = ???
display(joined_df)

Now, use aggregation to find the the min, max and avg donation by gender.

In [29]:
donations_by_gender = joined_df.???
donations_by_gender.show()

Now, make the necessary transformations and aggregations to answer to the following questions about the data. Note that some questions are only about the users, so make sure to use the smaller possible dataset when looking for your answers!

**Question 1:** 

a) What's the average, min and max age of the users? 

b) What's the average, min and max age of the users, by gender?

In [31]:
result_1a = ???
result_1a.show()

result_1b = ???
result_1b.show()

**Question 2:**

a) How many distinct country origins exist in the data? Print a DataFrame listing them.

b) What are the top 5 countries with the most users? Print a DataFrame containing the name of the countries and the counts.

In [33]:
result_2a = ???
result_2a.show()

result_2b = ???
result_2b.show()

**Question 3:**

What's the number of donations average, min and max donations values by age range?

In [35]:
result_3 = ???
result_3.show()

**Question 4:**

a) What's the number of donations, average, min and max donation values by user location (country)?

b) What is the number of donations, average, min and max donation values by gender for each user location (contry)? (the resulting DataFrame must contain 5 columns: the gender of the user, their country and the 3 metrics)

In [37]:
result_4a = ???
result_4a.show()

result_4b = ???
result_4b.show()

**Question 5**

Which month of the year has the largest aggregated donation value?

**Hint**: you can use a function to extract the month from a date, then you can aggregate to find the total donation value.

In [39]:
result_5 = ???

### 3. Window Functions

_This section uses the same data as the last one._

Window functions are very useful for gathering aggregated data without actually aggregating the DataFrame. They can also be used to find "previous" and "next" information for entities, such as an user. [This article](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) has a very nice explanation about the concept.

We want to find the users who donated less than a threshold and remove them from the donations dataset. Now, there are two ways of doing that:

1) Performing a traditional aggregation to find the users who donated less than the threshold, then filtering these users from the donations dataset, either with `where(not(df("user_id").isin(a_local_list)))` or with a join of type "anti-join".

2) Using window functions to add a new column with the aggregated donations per user, and then using a normal filter such as `where(aggregated_donation < threshold)`

Let's implement both and compare the complexity:

First, perform the traditional aggregation and find the users who donated less than 500 in total:

In [42]:
bad_users = joined_df.???
bad_users.show()

You should have found around 10 users. Now, perform an "anti-join" to remove those users from `joined_df`.

**Hint:** The `join` operation accepts a third argument which is the join type. The accepted values are: 'inner', 'outer', 'full', 'fullouter', 'leftouter', 'left', 'rightouter', 'right', 'leftsemi', 'leftanti', 'cross'.

In [44]:
good_donations = joined_df.???
good_donations.count()

Verify if the count of `good_donations` makes sense by performing a normal join to find the `bad_donations`.

In [46]:
bad_donations = joined_df.???
bad_donations.count()

If you done everything right, at this point `good_donations.count()` + `bad_donations.count()` = `joined_df.count()`.

But using the join approach can be very heavy and it requires multipe operations. For this kind of problems, Window Functions are better.

In [48]:
good_donations.count() + bad_donations.count() == joined_df.count()

The first step is to create your window specification over `user_id` by using partitionBy

In [50]:
from pyspark.sql import Window

window_spec = Window.???

Then, you can use one of the window functions of the `pyspark.sql.functions` package, appling to the created window_spec by using the `over` method. 

**Hint:** If you are blocked, try looking at the [documentation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.over) for the `over` method, or searching on StackOverflow.

In [52]:
new_column = ???.over(???)

donations_with_total = joined_df.???
donations_with_total.show()

And now you can just filter on the `total_donated_by_user` column:

In [54]:
good_donations_wf = donations_with_total.???
good_donations_wf.count()

If you done everything right, you should obtain `good_donations_wf.count()` = `good_donations.count()`

In [56]:
good_donations_wf.count() == good_donations.count()

Window functions also can be useful to find the "next" and "previous" operations by using `functions.lead` and `functions.lag`. In our example, we can use it to find the date interval between two donations by a specific user.

For this kind of Window function, the window specification must be ordered by the date. So, create a new window specification partitioned by the `user_id` and ordered by the donation timestamp.

In [58]:
ordered_window = Window.???

Now use `functions.lag().over()` to add a column with the timestamp of the previous donation of the same user. Then, inspect the result to see what it looks like.

In [60]:
new_column = ???.over(???)

donations_with_lag = good_donations.???
donations_with_lag.orderBy("user_id", "timestamp").show()

Finally, compute the average time it took for each user between two of their consecutive donations (in days), and print the 5 users with the smallest averages. The result must include at least the users' id, last name and birth date, as well as the computed average.

In [62]:
users_average_between_donations = donations_with_lag.???
users_average_between_donations.show(5)

Congratulations, you have finished this notebook!